# Tracking the Sun

Berkeley Lab’s *Tracking the Sun* report series is dedicated to summarizing installed prices and other trends among grid-connected, distributed solar photovoltaic (PV) systems in the United States. For more information, please refer to https://emp.lbl.gov/tracking-the-sun/

## 0. Build Database Connection

PyAthena is a Python DB API 2.0 (PEP 249) compliant client for the Amazon Athena JDBC driver.
https://github.com/laughingman7743/PyAthena

In [1]:
from pyathena.connection import Connection
from pyathena.pandas_cursor import PandasCursor

In [2]:
AWS_REGION_NAME = "us-west-2"
DATABASE_NAME = "oedidb"
TABLE_NAME = "tracking_the_sun"
S3_STAGING_DIR = "s3://nrel-tests/tracking-the-sun-staging"

In [3]:
cursor = Connection(region_name=AWS_REGION_NAME, s3_staging_dir=S3_STAGING_DIR).cursor()

In [4]:
pandas_cursor = Connection(region_name=AWS_REGION_NAME, s3_staging_dir=S3_STAGING_DIR).cursor(PandasCursor)

## 1. Retrieve Table Metadata

### 1.1 Columns
Retrieve the schema

In [5]:
import pandas as pd

In [6]:
# Retrieve column information
result = cursor.execute(f"DESCRIBE {DATABASE_NAME}.{TABLE_NAME}")
columns = [[item.strip() for item in row[0].split("\t")] for row in result.fetchall()]
pd.DataFrame(columns, columns=["NAME", "TYPE", "FROM"])

,NAME,TYPE,FROM
0,data_provider,varchar(120),
1,system_id_from_data_provider,varchar(120),
2,system_id_tracking_the_sun,varchar(120),
3,installation_date,date,
4,system_size,double,
5,total_installed_price,double,
6,appraised_value_flag,boolean,
7,sales_tax_cost,double,
8,rebate_or_grant,double,
9,performance_based_incentive_annual_payment,double,


### 1.2 Partitions
Retrieve the partitions, partition key is *state*

In [7]:
# Retrieve parition information
result = cursor.execute(f"SHOW PARTITIONS {DATABASE_NAME}.{TABLE_NAME}")
for row in result.fetchall():
    print(row)

('state=NH',)
('state=OH',)
('state=NM',)
('state=DE',)
('state=AR',)
('state=TX',)
('state=MO',)
('state=UT',)
('state=ME',)
('state=AZ',)
('state=-9999',)
('state=MA',)
('state=IL',)
('state=NY',)
('state=MN',)
('state=CO',)
('state=CA',)
('state=KS',)
('state=VT',)
('state=WI',)
('state=MD',)
('state=DC',)
('state=OR',)
('state=FL',)
('state=CT',)
('state=PA',)


## 2. PV System Installation Trend
To visualize the number of PV System installation among states using grid plot.

In [8]:
# bokeh
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, PrintfTickFormatter, ColorBar, LogColorMapper, LogTicker
from bokeh.plotting import figure, show
output_notebook()

Loading BokehJS ...

In [9]:
pv_state_year = pandas_cursor.execute(
    f"""
    SELECT state, CAST(YEAR(installation_date) AS VARCHAR(4)) AS year, COUNT(*) as count
    FROM {DATABASE_NAME}.{TABLE_NAME}
    GROUP BY CAST(YEAR(installation_date) AS VARCHAR(4)), state;
    """
).as_pandas()

In [10]:
years = sorted(pv_state_year["year"].unique())
states = list(reversed(sorted(pv_state_year["state"].unique())))

ifig = figure(
    title="Heatmap of State PV Systems",
    x_range=years,
    y_range=states,
    plot_height=350,
    sizing_mode="scale_width",
    x_axis_location="above",
    toolbar_location='below',
    tooltips=[("state", "@state"), ("year", "@year"), ("count", "@count")]
)

colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LogColorMapper(
    palette=colors,
    low=pv_state_year["count"].min(),
    high=pv_state_year["count"].max()
)
ifig.rect(
    x="year",
    y="state",
    width=1,
    height=1,
    source=pv_state_year,
    fill_color={"field": "count", "transform": mapper},
    line_color=None
)
ifig.grid.grid_line_color = None
ifig.axis.axis_line_color = None
ifig.axis.major_tick_line_color = None
ifig.axis.major_label_text_font_size = "6pt"
ifig.axis.major_label_standoff = 0

color_bar = ColorBar(
    color_mapper=mapper,
    major_label_text_font_size="6pt",
    ticker=LogTicker(),
    formatter=PrintfTickFormatter(format="%d"),
    label_standoff=6,
    border_line_color=None,
    location=(0, 0),
)

ifig.add_layout(color_bar, "right")


show(ifig)

## 3. PV System Geographic Distribution
To visualize the number of PV system installation in US at *state*, *county* and *city* level.

In [11]:
import math
from branca import colormap
from branca.element import Template, MacroElement
import folium
from folium import plugins
import geopandas
from shapely import geometry

### 3.1 State Level Distribution

In [12]:
tts = "Tracking-the-Sun"
# Query database and load data into geodataframe.
pv_state = pandas_cursor.execute(
    f"""
    SELECT state, COUNT(*) AS count 
    FROM {DATABASE_NAME}.{TABLE_NAME} 
    GROUP by state;
    """
).as_pandas()
geo_state = geopandas.read_file(f"{tts}/us-states.geojson")[["id", "geometry"]]
geo_state.rename(columns={"id": "state"}, inplace=True)

geo_pv_state = geopandas.GeoDataFrame(
    data=pv_state.merge(geo_state, on="state"),
    geometry="geometry",
    crs={"init": "epsg:4326"}
)

In [13]:
# Display state pv system numbers on map
state_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="OpenStreetMap")

tooltip = folium.GeoJsonTooltip(
    fields=["count"],
    aliases=["count:"],
    labels=True,
    sticky=False
)

colors = ["#ffffcc","#ffeda0","#fed976","#feb24c","#fd8d3c","#fc4e2a","#e31a1c","#bd0026","#800026"]
bins = [0, 100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000]
colorscale = colormap.StepColormap(
    colors=colors,
    index=bins,
    vmin=0,
    vmax=1000000
)
def style_function(feature):
    count = feature["properties"]["count"]
    return {
        "color": "#000000",
        "weight": 0.2,
        "opacity": 0.6,
        "fillColor": colorscale(count),
        "fillOpacity": 0.4,
    }

folium.GeoJson(
    name="Distribution of State PV Systems",
    data=geo_pv_state.to_json(),
    tooltip=tooltip,
    style_function=style_function
).add_to(state_map)

legend = MacroElement()
with open(f"{tts}/pv_distribution_legend.html") as f:
    template = f.read()
legend._template = Template(template)
state_map.get_root().add_child(legend)

state_map

### 3.2 County Level Distribution

In [14]:
# Query database and load data into geodataframe.
pv_county = pandas_cursor.execute(
    f"""
    SELECT state, county, COUNT(*) AS count
    FROM {DATABASE_NAME}.{TABLE_NAME}
    GROUP by state, county;
    """
).as_pandas()

# convert pandas dataframe to geodataframe
geo_county = geopandas.read_file(f"{tts}/us-counties.geojson")[["state", "county", "geometry"]]

# WARNING: pv_county contains invalid county name '-9999', use inner join here.
geo_pv_county = geopandas.GeoDataFrame(
    data=pd.merge(pv_county, geo_county, how="inner", left_on=["state", "county"], right_on=["state", "county"]),
    geometry="geometry",
    crs={"init": "epsg:4326"}
)

In [15]:
# Display county pv system numbers on map
county_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="OpenStreetMap")

tooltip = folium.GeoJsonTooltip(
    fields=["count"],
    aliases=["count:"],
    labels=True,
    sticky=False
)

colorscale = colormap.LinearColormap(
    colors=["#ffffcc","#ffeda0","#fed976","#feb24c","#fd8d3c","#fc4e2a","#e31a1c","#bd0026","#800026"],
    index=[100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000]
)
def style_function(feature):
    count = feature["properties"]["count"]
    return {
        "color": "#000000",
        "weight": 0.2,
        "opacity": 0.6,
        "fillColor": colorscale(count),
        "fillOpacity": 0.4,
    }

folium.GeoJson(
    name="Distribution of State PV Systems",
    data=geo_pv_county.to_json(),
    tooltip=tooltip,
    style_function=style_function
).add_to(county_map)

legend = MacroElement()
with open(f"{tts}/pv_distribution_legend.html") as f:
    template = f.read()
legend._template = Template(template)
county_map.get_root().add_child(legend)

county_map

### 3.3 City Level Distribution

In [16]:
pv_city = pandas_cursor.execute(
    f"""
    SELECT state, county, city, COUNT(*) AS count
    FROM {DATABASE_NAME}.{TABLE_NAME}
    GROUP by state, county, city;
    """
).as_pandas()

geo_city = geopandas.read_file(f"{tts}/us-cities.geojson")[["state", "county", "city", "geometry"]]

# Handle case
pv_city["city"] = pv_city["city"].str.title()
pv_city["county"] = pv_city["county"].str.title()

geo_city["city"] = geo_city["city"].str.title()
geo_city["county"] = geo_city["county"].str.title()

# Merge
geo_pv_city = geopandas.GeoDataFrame(
    data=pd.merge(pv_city, geo_city, how="inner", left_on=["state", "county", "city"], right_on=["state", "county", "city"]),
    geometry="geometry",
    crs={"init": "epsg:4326"}
)

In [17]:
# Display city pv system numbers on map
city_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="cartodbpositron")

# Marker Cluster
coordinates = [[geom.y, geom.x] for geom in geo_pv_city.geometry.values]
marker_cluster = plugins.FastMarkerCluster(data=coordinates)
marker_cluster.add_to(city_map)

# Heatmap
heat_map = plugins.HeatMap(coordinates, radius=15)
heat_map.add_to(city_map)

city_map

## 4. PV System Unit Price Trend
To visualize the average installation price of PV system over past years state by state.

In [18]:
import numpy as np
from ipywidgets import interact
from bokeh.core.properties import value
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap

In [19]:
pv_price = pandas_cursor.execute(
    f"""
    SELECT 
        state,
        CAST(YEAR(installation_date) AS VARCHAR(4)) AS year,
        system_size,
        total_installed_price,
        ROUND((total_installed_price / system_size), 2) AS unit_installed_price,
        customer_segment
    FROM {DATABASE_NAME}.{TABLE_NAME}
    WHERE total_installed_price != -9999
    AND system_size != -9999 
    AND system_size != 0
    """
).as_pandas()

In [20]:
pv_price.head()

,state,year,system_size,total_installed_price,unit_installed_price,customer_segment
0,ME,2012,4.41,15109.0,3426.08,-9999
1,ME,2012,7.20,44950.0,6243.06,-9999
2,ME,2012,4.80,16175.0,3369.79,-9999
3,ME,2012,5.76,18155.0,3151.91,-9999
4,ME,2012,4.50,18375.0,4083.33,-9999


In [21]:
states = sorted(pv_price["state"].unique())
customers = pv_price["customer_segment"].unique()

In [22]:
@interact
def show_pv_price_trend(state=states):
    # filter
    state_pv_price = pv_price[pv_price["state"]==state]
    
    # wrapping
    unstack_pv_price = state_pv_price.groupby(["customer_segment", "year"]).mean().round(2)[["unit_installed_price"]].unstack(0)
    customer_pv_price = unstack_pv_price.reindex(pd.Index([str(x) for x in list(range(1998, 2018, 1))], name="year"))["unit_installed_price"]
    
    customers = ["RES", "NON-RES"]
    for customer in customers:
        if customer in customer_pv_price.columns:
            continue
        customer_pv_price.loc[:, customer] = np.NaN
    customer_pv_price = customer_pv_price[customers] #.fillna(0)
    
    # plots
    years = customer_pv_price.index.values
    categories = ["R", "N"]
    data = {
        "years": years,
        "R": customer_pv_price["RES"],
        "N": customer_pv_price["NON-RES"]
    }
    x = [(year, customer) for year in years for customer in categories]
    prices = sum(zip(data["R"], data["N"]), ())
    
    fig = figure(
        title="PV Unit Price Trend",
        x_range=FactorRange(*x),
        plot_height=300,
        sizing_mode="scale_width",
        tools="hover",
        tooltips="@prices",
        toolbar_location="below"
    )
    
    source = ColumnDataSource(data=dict(x=x, prices=prices))
    colors = ["#718dbf", "#e84d60"]
    fig.vbar(
        source=source,
        x="x",
        top='prices',
        width=0.9,
        fill_color=factor_cmap('x', palette=colors, factors=categories, start=1, end=2),
        line_color=factor_cmap('x', palette=colors, factors=categories, start=1, end=2),
        alpha=0.7
    )
    fig.line(x=years, y=data["R"], line_width=2, line_color="#718dbf")
    fig.line(x=years, y=data["N"], line_width=2, line_color="#e84d60")
    
    fig.xgrid.grid_line_color = None
    show(fig)

interactive(children=(Dropdown(description='state', options=('AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'IL', '…

R: Residential <br>
N: Non-residential

In [23]:
state = "NY"
state_pv_price = pv_price[pv_price["state"]==state]
    
# wrapping
unstack_pv_price = state_pv_price.groupby(["customer_segment", "year"]).mean().round(2)[["unit_installed_price"]].unstack(0)
customer_pv_price = unstack_pv_price.reindex(pd.Index([str(x) for x in list(range(1998, 2018, 1))], name="year"))["unit_installed_price"]

customers = ["RES", "NON-RES"]
for customer in customers:
    if customer in customer_pv_price.columns:
        continue
    customer_pv_price.loc[:, customer] = np.NaN
customer_pv_price = customer_pv_price[customers] #.fillna(0)

# plots
years = customer_pv_price.index.values
categories = ["R", "N"]
data = {
    "years": years,
    "R": customer_pv_price["RES"],
    "N": customer_pv_price["NON-RES"]
}
x = [(year, customer) for year in years for customer in categories]
prices = sum(zip(data["R"], data["N"]), ())

fig = figure(
    title="PV Unit Price Trend",
    x_range=FactorRange(*x),
    plot_height=300,
    sizing_mode="scale_width",
    tools="hover",
    tooltips="@prices",
    toolbar_location="below"
)

source = ColumnDataSource(data=dict(x=x, prices=prices))
colors = ["#718dbf", "#e84d60"]
fig.vbar(
    source=source,
    x="x",
    top='prices',
    width=0.9,
    fill_color=factor_cmap('x', palette=colors, factors=categories, start=1, end=2),
    line_color=factor_cmap('x', palette=colors, factors=categories, start=1, end=2),
    alpha=0.7
)
fig.line(x=years, y=data["R"], line_width=2, line_color="#718dbf")
fig.line(x=years, y=data["N"], line_width=2, line_color="#e84d60")

fig.xgrid.grid_line_color = None
show(fig)